In [1]:
import pandas as pd
import os
import math
import numpy as np
import networkx as nx
import my_nx as my_nx
import matplotlib.pyplot as plt
from datetime import datetime,time, timedelta
import pickle
from process_transfers import get_merged_stops, get_merged_stops_names

In [2]:
def get_data_paths(main_dir):
    data_paths = []
    for root, dirs, files in os.walk(main_dir):
        for dir in dirs:
            data_paths.append(os.path.join(root, dir))
        break
    
    return data_paths
            
paths = get_data_paths(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data')
paths

['C:\\Users\\baodu\\Dropbox\\Summer_research_2024\\MTA_data\\google_transit._MTA_bus_company',
 'C:\\Users\\baodu\\Dropbox\\Summer_research_2024\\MTA_data\\google_transit_bronx',
 'C:\\Users\\baodu\\Dropbox\\Summer_research_2024\\MTA_data\\google_transit_brooklyn',
 'C:\\Users\\baodu\\Dropbox\\Summer_research_2024\\MTA_data\\google_transit_manhattan',
 'C:\\Users\\baodu\\Dropbox\\Summer_research_2024\\MTA_data\\google_transit_queens',
 'C:\\Users\\baodu\\Dropbox\\Summer_research_2024\\MTA_data\\google_transit_staten_island']

In [5]:
trips_lst = []
stop_times_lst = []
stops_lst = []

for base_path in paths:
    trips_lst.append(pd.read_csv(f'{base_path}\\updated_trips.txt'))
    stop_times_lst.append(pd.read_csv(f'{base_path}\\stop_times.txt'))
    stops_lst.append(pd.read_csv(f'{base_path}\\stops.txt'))

# print(stops_lst)

trips = pd.concat(trips_lst, ignore_index=True)
stop_times = pd.concat(stop_times_lst, ignore_index=True)
stops = pd.concat(stops_lst, ignore_index=True)

stop_lst = stops['stop_id'].unique()
stop_lst

array([100025, 100027, 100033, ..., 905220, 905221, 905222], dtype=int64)

In [6]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

In [7]:
stops_mapping={}

In [8]:
G = nx.Graph()

for index, row in stops.iterrows():
    G.add_node(row['stop_id'], pos=(row['stop_lat'], row['stop_lon']), name=row['stop_name'])
    
def merge_close_nodes(G, threshold):
    pos = nx.get_node_attributes(G, 'pos')
    names = nx.get_node_attributes(G, 'name')
    nodes = list(G.nodes())
    print(nodes)
    clusters = []
    visited = set()
    

    for node in nodes:
        if node not in visited:
            cluster = {node}
            queue = [node]
            visited.add(node)
            while queue:
                current = queue.pop(0)
                current_pos = G.nodes[current]['pos']
                for neighbor in nodes:
                    if neighbor not in visited:
                        neighbor_pos = G.nodes[neighbor]['pos']
                        if haversine(current_pos[0], current_pos[1], neighbor_pos[0], neighbor_pos[1]) < threshold:
                            visited.add(neighbor)
                            queue.append(neighbor)
                            cluster.add(neighbor)
            clusters.append(cluster)
    
    for cluster in clusters:
        for node in cluster:
            str_val = ' '.join(list(map(str, cluster)))
            stops_mapping[node] = str_val
    
    return clusters

clusters = merge_close_nodes(G, 200)
print(clusters)

[100025, 100027, 100033, 100039, 100045, 100047, 100048, 100058, 100060, 100071, 100077, 100088, 100089, 100098, 100100, 100101, 100103, 100109, 100114, 100131, 100134, 100166, 100183, 100294, 100296, 100297, 100298, 100322, 100325, 100328, 100330, 100332, 100459, 100461, 100462, 100463, 100464, 100466, 100467, 100469, 100471, 100472, 100473, 100474, 100483, 100588, 100589, 100590, 100591, 100592, 100594, 100595, 100609, 100610, 100612, 100613, 100614, 100615, 100676, 100677, 100678, 100679, 100700, 100706, 100707, 100730, 100731, 100821, 100823, 100824, 100826, 100828, 100829, 100860, 100861, 100863, 100865, 100866, 100868, 101137, 101138, 101140, 101142, 101190, 101192, 101193, 101194, 101415, 101418, 101420, 101422, 101423, 101440, 101444, 101447, 101716, 101717, 101718, 101719, 101721, 101722, 101723, 101724, 101725, 101726, 101727, 101728, 101729, 101730, 101731, 101734, 101735, 101737, 101738, 101740, 101742, 101743, 101744, 101745, 101746, 101747, 101749, 101750, 101751, 101752,

In [9]:
print(len(clusters))
# print(stops_mapping)

1411


In [10]:
stop_details = pd.merge(stop_times, trips, on='trip_id')
stop_details = pd.merge(stop_details, stops, on='stop_id')
stop_details = stop_details.sort_values(by=['trip_id', 'stop_sequence'])
# stop_details.to_csv(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data\stop_details.txt')

In [11]:
stop_details['stop_id'] = stop_details.apply(lambda row: stops_mapping[row['stop_id']] if row['stop_id'] in stops_mapping else row['stop_id'], axis=1)

stops['stop_id'] = stops.apply(lambda row: stops_mapping[row['stop_id']] if row['stop_id'] in stops_mapping else row['stop_id'], axis=1)
# stops['stop_id'] = stops['stop_id'].map(stops_mapping)
stop_lst = stops['stop_id'].unique()
print(stop_lst)

['100366 102166 102167 102168 102169 102170 103968 100133 100134 100135 802088 104116 100025 802146 102115 102116 102117 102118 102119'
 '100131 100027'
 '103435 100749 102159 100112 100113 100114 100115 102162 103951 902169 902170 100768 103969 100769 103073 102821 103974 103211 103212 103214 103215 150063 102451 100033 100035 102729 102730 102994 102357 103781 902119 902120 102123 102127 102128 104049 102130 102131 802165 103798'
 ... '905154 905155' '905179 905180' '905221']


In [12]:
print(len(stop_lst))

1411


In [13]:
# Find routes per stop
route_per_stop = {}
for row in stop_details.itertuples():
    if row.stop_id not in route_per_stop:
        route_per_stop[row.stop_id] = [row.route_id]
    else:
        if row.route_id not in route_per_stop[row.stop_id]:
            route_per_stop[row.stop_id].append(row.route_id)
        else:
            continue
# route_per_stop
stop_details['routes_per_stop'] = stop_details['stop_id'].map(route_per_stop)
print(route_per_stop)

{'982081 551784 551785 551786 551787': ['Q65'], '551788 551789': ['Q65'], '551047 551048 505125 505017 505018 502459 502460 505019 502462 505020 505024 505025 505021 505022 505023 982078 502464 502465 505040 505044 505045 551776 551020 551021 551022 551023 551024 551025 551026 551027 551028 551029': ['Q65', 'Q25', 'Q20A', 'Q76', 'Q20B'], '551771 551772': ['Q65'], '552965 551766 551767 551768 551769 551770': ['Q65'], '553300 553301': ['Q65'], '551764 551765': ['Q65'], '552979 551763': ['Q65'], '551793 551762': ['Q65'], '551761 500994 502254': ['Q65', 'Q66', 'Q19', 'Q34', 'Q25', 'Q50', 'Q20A', 'Q20B'], '503845 501811 904298 904299 501871 501360 501361 501362 501363 500851 500852 500853 501367 500854 501368 501370 501371 501369 501365 501372 501373 501374 551050 551051 504476 504477 553117 504992 504993 504994 504487 804017 804031 904384 500931 500932 804041 804042 503501 505048 804059 504034 904946 504081 501530 501531 501532 501533 501534 501535 501536 505121 503074 501537 501538 501539

In [14]:
def parse_time(curr_time):
    if curr_time == '24:00:00':
        return pd.to_datetime('2024-01-01 00:00:00')
    else:
        try:
            return pd.to_datetime(time, format='%H:%M:%S')
        
        except ValueError:
            h, m, s = map(int, curr_time.split(':'))
            h = h % 24
            return (datetime(2024, 1, 1, h, m, s) + timedelta(days=h // 24))

stop_details['departure_time'] = stop_details['departure_time'].apply(parse_time)
stop_details['arrival_time'] =stop_details['arrival_time'].apply(parse_time)

stop_details_weekday = stop_details[stop_details['service_id'].str.contains(r'Weekday', na=False)]
stop_details_weekends = stop_details[stop_details['service_id'].str.contains(r'Saturday', na=False) | stop_details['service_id'].str.contains(r'Sunday', na=False)]
# stop_details_weekends

In [15]:
def build_matrix(stop_details):
    trip_count = pd.DataFrame(0, index=stop_lst, columns=stop_lst)
    travel_time = pd.DataFrame(0.0, index=stop_lst, columns=stop_lst)
    for i in range(len(stop_details)-1):
        curr_seq = stop_details.iloc[i]['stop_sequence']
        curr_id = stop_details.iloc[i]['stop_id']
        next_seq = stop_details.iloc[i+1]['stop_sequence']
        next_id = stop_details.iloc[i+1]['stop_id']
                
        if curr_seq+1 == next_seq:
            trip_count.loc[curr_id, next_id] += 1
            travel_time.loc[curr_id, next_id] = (stop_details.iloc[i+1]['arrival_time'] - stop_details.iloc[i]['departure_time']).total_seconds() / 60
                
    return trip_count, travel_time

# build_matrix(stop_details_weekday)

In [16]:
def compute_edge_labels(trip_count, travel_time):
    edge_labels ={}
    for i in trip_count.index:
        for j in trip_count.columns:
            if trip_count.loc[i,j] !=0:
                edge_labels[(i, j)] = (trip_count.loc[i,j],  travel_time.loc[i,j])
    return edge_labels

In [17]:
def build_network(clusters, trip_count, travel_time, title):
    # Create a new graph with merged nodes
    new_G = nx.DiGraph()
    names = nx.get_node_attributes(G, 'name')
    nodes = list(G.nodes())
    
    for cluster in clusters:
        # Calculate centroid
        lat_sum = sum(G.nodes[node]['pos'][0] for node in cluster)
        lon_sum = sum(G.nodes[node]['pos'][1] for node in cluster)
        centroid = (lon_sum / len(cluster), lat_sum / len(cluster))
        
        # Combine names
        combined_name = " / ".join(sorted({names[node] for node in cluster}))
        temp_name = ' '.join(list(map(str, cluster)))
        new_node = f"{temp_name}"
        new_G.add_node(new_node, pos=centroid, name=combined_name, routes=route_per_stop[temp_name])

    # print(nx.get_node_attributes(new_G,'routes'))
    
    for i in trip_count.index:
        for j in trip_count.columns:
            if trip_count.loc[i, j] > 0:  # There is a trip from i to j
                routes_i = set(nx.get_node_attributes(new_G, 'routes')[i])
                routes_j = set(nx.get_node_attributes(new_G, 'routes')[j])
                # Ensure both stops share at least one common route
                common_routes = routes_i.intersection(routes_j)
                if common_routes:
                    new_G.add_edge(i, j, vechicle_trips=trip_count.loc[i, j], travel_time=travel_time.loc[i, j], routes=list(common_routes))
    

    new_G.remove_edges_from(nx.selfloop_edges(new_G))

    pos = nx.get_node_attributes(new_G, 'pos')
    name = nx.get_node_attributes(new_G, 'name')
    edge_labels = {(u, v): f"{d['vechicle_trips']}, {d['travel_time']}" for u, v, d in new_G.edges(data=True)}

    
    # Draw the graph
    plt.figure(figsize=(40, 30))
    nx.draw(new_G, pos, node_size=8, node_color='grey', labels=name, font_color='purple', font_size='12', with_labels=False, arrowsize=20, verticalalignment='bottom', connectionstyle='arc3, rad = 0.1')
    
    my_nx.my_draw_networkx_edge_labels(new_G, pos=pos, edge_labels=edge_labels, font_color='red', font_size=5, rotate=True, rad=0.1)

    plt.show()
    
    # plt.savefig(f'{title}', dpi=500)
    pickle.dump(new_G, open(f'{title}.pickle', 'wb'))
    
    return new_G

In [18]:
%matplotlib widget
%matplotlib qt

In [19]:
weekday_trip_count, weekday_travel_time = build_matrix(stop_details_weekday)
weekend_trip_count, weekend_travel_time = build_matrix(stop_details_weekends)


weekday_trip_count.to_csv(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data\weekday_trip_count.csv')
weekend_trip_count.to_csv(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data\weekend_trip_count.csv')

weekday_travel_time.to_csv(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data\weekday_travel_time.csv')
weekend_travel_time.to_csv(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data\weekend_travel_time.csv')


# weekday_trip_count = pd.read_csv(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data\weekday_trip_count.csv')
# weekend_trip_count = pd.read_csv(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data\weekend_trip_count.csv')
# # print(weekday_trip_count)

# weekday_travel_time = pd.read_csv(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data\weekday_travel_time.csv')
# weekend_travel_time = pd.read_csv(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data\weekend_travel_time.csv')

weekday_edge_labels = compute_edge_labels(weekday_trip_count, weekday_travel_time)
weekend_edge_labels = compute_edge_labels(weekend_trip_count, weekend_travel_time)

In [20]:
# weekday_trip_count.to_csv(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data\weekday_trip_count.csv')
# weekend_trip_count.to_csv(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data\weekend_trip_count.csv')

# weekday_travel_time.to_csv(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data\weekday_travel_time.csv')
# weekend_travel_time.to_csv(r'C:\Users\baodu\Dropbox\Summer_research_2024\MTA_data\weekend_travel_time.csv')

In [22]:
# weekday_G = build_network(clusters, weekday_trip_count, weekday_travel_time, 'bus_network_weekday')
weekend_G = build_network(clusters, weekend_trip_count, weekend_travel_time, 'bus_network_weekend')


# print(weekday_G)

In [ ]:
nx.single_source_dijkstra_path(weekday_G, '101', weight='travel_time')

NodeNotFound: Node 101 not found in graph

In [ ]:

        
def print_shortest_paths(G, source):
    shortest_paths = dict(nx.all_pairs_dijkstra_path_length(G, weight='travel_time'))
    paths = nx.single_source_dijkstra_path(G, source, weight='travel_time')
    for target, time in shortest_paths[source].items():
        path = paths[target]
        route_info = []
        stop_info = []
        transfer_info = []
        previous_routes = None
        segment_start = path[0]
        
        for i in range(len(path) - 1):
            u, v = path[i], path[i + 1]
            routes = set(G[u][v]['routes'])
            if previous_routes is None:
                previous_routes = routes
            elif not set(previous_routes).issubset(set(routes)) and not set(routes).issubset(set(previous_routes)):
                route_info.append(f"{segment_start} -> {u} via routes {previous_routes}")
                transfer_info.append(f"Transfer at {u} from routes {previous_routes} to routes {routes}")
                segment_start = u
                previous_routes = routes
            stop_info.append(f"{u}")
        stop_info.append(f"{target}")  # Include the final stop
        route_info.append(f"{segment_start} -> {target} via routes {previous_routes}")
        print(f"Shortest travel time from {source} to {target}: {time} minutes.")
        print(f"Stops: {' -> '.join(stop_info)}")
        print(f"Routes: {', '.join(route_info)}")
        if transfer_info:
            print(f"Transfers: {'; '.join(transfer_info)}")
        print()

print_shortest_paths(weekday_G, '101')

Shortest travel time from 101 to 101: 0 minutes.
Stops: 101
Routes: 101 -> 101 via routes None

Shortest travel time from 101 to 103: 1.5 minutes.
Stops: 101 -> 103
Routes: 101 -> 103 via routes {'1'}

Shortest travel time from 101 to 104: 3.0 minutes.
Stops: 101 -> 103 -> 104
Routes: 101 -> 104 via routes {'1'}

Shortest travel time from 101 to 106: 4.5 minutes.
Stops: 101 -> 103 -> 104 -> 106
Routes: 101 -> 106 via routes {'1'}

Shortest travel time from 101 to 107: 6.0 minutes.
Stops: 101 -> 103 -> 104 -> 106 -> 107
Routes: 101 -> 107 via routes {'1'}

Shortest travel time from 101 to 108: 7.0 minutes.
Stops: 101 -> 103 -> 104 -> 106 -> 107 -> 108
Routes: 101 -> 108 via routes {'1'}

Shortest travel time from 101 to 109: 8.5 minutes.
Stops: 101 -> 103 -> 104 -> 106 -> 107 -> 108 -> 109
Routes: 101 -> 109 via routes {'1'}

Shortest travel time from 101 to 110: 10.0 minutes.
Stops: 101 -> 103 -> 104 -> 106 -> 107 -> 108 -> 109 -> 110
Routes: 101 -> 110 via routes {'1'}

Shortest trave

In [ ]:
# dicts = nx.get_edge_attributes(weekday_G, 'travel_time')
# vals = []
# for key,val in dicts.items():
#     if val<0:
#         print(f'{key}: {val}')
#     vals.append(val)
# print(vals)

# shortest_paths = dict(nx.all_pairs_dijkstra_path_length(weekday_G, weight='travel_time'))

# # Example: Print shortest travel times from stop '101S' to all other stops
# for target, time in shortest_paths['101'].items():
#     print(f"Shortest travel time from 101 to {target}: {time} minutes")

In [ ]:
def visualize_paths(G, source, target):
    shortest_paths = dict(nx.all_pairs_dijkstra_path_length(G, weight='travel_time'))
    paths = nx.single_source_dijkstra_path(G, source, weight='travel_time')
    
    path = paths[target]
    plt.figure(figsize=(20, 15))
    
    # Draw the entire graph
    pos = nx.get_node_attributes(G, 'pos')
    name = nx.get_node_attributes(G, 'name')
    nx.draw(G, pos, node_size=8, node_color='grey', labels=name, font_color='purple', font_size='12', with_labels=True, arrowsize=20, verticalalignment='top', connectionstyle='arc3, rad=0.1')
    
    # Highlight the path
    path_edges = [(path[i], path[i + 1]) for i in range(len(path) - 1)]
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='blue', width=2.5)
    nx.draw_networkx_nodes(G, pos, nodelist=path, node_color='blue', node_size=50)
    
    plt.title(f"Paths from {source} to {target}")
    plt.show()

visualize_paths(weekday_G, '101', 'H11')